In [1]:
## import 
# import sys
import os
import numpy as np
import pandas as pd
import boto3
from botocore.exceptions import NoCredentialsError
from dotenv import load_dotenv

In [2]:
## variables
# print(sys.executable)
seed = 0
ratio_train = 0.8
np.random.seed(seed)

In [ ]:
## data preprocess
df = pd.read_csv('inputs/weatherHistory.csv')
df['Date'] = pd.to_datetime(df['Formatted Date'], utc=True)
df['Temperature'] = df['Temperature (C)']
tdf = df[['Date', 'Temperature']]
tdf.fillna(method='ffill', inplace=True)
train_size = int(ratio_train * int(tdf.shape[0]))
train_df = tdf[:train_size]
test_df = tdf[train_size:]
train_df.to_csv('inputs/train.csv', index=False)
test_df.to_csv('inputs/test.csv', index=False)

In [ ]:
## upload to cloud
load_dotenv()
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')

def upload_to_s3(local_file, bucket, s3_file):
    """
    Upload a file to an AWS S3 bucket.
    
    :param local_file: File to upload.
    :param bucket: Bucket to upload to.
    :param s3_file: S3 object name. If not specified, same as local_file.
    """
    s3 = boto3.client(
        's3', 
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        region_name='us-east-2'
    )

    try:
        s3.upload_file(local_file, bucket, s3_file)
        print(f"Upload Successful: {local_file} to {bucket}/{s3_file}")
    except FileNotFoundError:
        print(f"The file {local_file} was not found")
    except NoCredentialsError:
        print("Credentials not available")

upload_to_s3('inputs/train.csv', 'mle-e2e-2', 'inputs/train.csv')
upload_to_s3('inputs/test.csv', 'mle-e2e-2', 'inputs/test.csv')
